<a href="https://colab.research.google.com/github/appersaravanan/final_project/blob/master/God_grace1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import densenet
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
from keras import backend as K
from keras.applications import VGG16
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import  MaxPooling2D,UpSampling2D,Input,BatchNormalization,Conv2D,Activation,MaxPool2D,Flatten,Dense
from tensorflow.keras.models import Model,Sequential,load_model
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt
from keras.utils import np_utils
from keras.preprocessing import image as image_utils
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras.applications import VGG16
import cv2,os,tempfile
import requests
from keras.preprocessing import image
from scipy.io import loadmat
import tensorflow as tf
import pandas as pd
import cv2
import matplotlib.pyplot as plt

In [55]:
img_width, img_height = 224, 224
nb_train_samples = 8144
nb_validation_samples = 4899
epochs = 10
batch_size = 64
n_classes = 196

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content
!mkdir final_project
%cd ./final_project
!mkdir data
%cd data

Mounted at /content/drive
/content
/content/final_project
/content/final_project/data


In [4]:

!rm -f car_devkit.tgz
!wget https://ai.stanford.edu/~jkrause/cars/car_devkit.tgz
!tar xvzf car_devkit.tgz

--2020-12-05 15:23:24--  https://ai.stanford.edu/~jkrause/cars/car_devkit.tgz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 330960 (323K) [application/x-gzip]
Saving to: ‘car_devkit.tgz’

car_devkit.tgz      100%[===================>] 323.20K   231KB/s    in 1.4s    

2020-12-05 15:23:26 (231 KB/s) - ‘car_devkit.tgz’ saved [330960/330960]

devkit/
devkit/cars_meta.mat
devkit/cars_train_annos.mat
devkit/cars_test_annos.mat
devkit/README.txt
devkit/train_perfect_preds.txt
devkit/eval_train.m


In [ ]:
!wget http://imagenet.stanford.edu/internal/car196/cars_train.tgz

!tar xvzf cars_train.tgz
!rm -f cars_train.tgz


In [6]:
%cd /content/final_project/data

/content/final_project/data


In [ ]:
!wget http://imagenet.stanford.edu/internal/car196/cars_test.tgz
!tar xvzf cars_test.tgz
!rm -f cars_test.tgz

In [ ]:
%cd /content/final_project/data/cars_train/
annotsa = loadmat('/content/final_project/data/devkit/cars_meta.mat')
i=1
classes_name=[]
for name in annotsa['class_names'][0]:
  dir_name=str(i)+'_'+str(name[0]).replace(' ','_')
  os.makedirs(dir_name)
  classes_name.append(dir_name)
  i=i+1
classes_name


In [ ]:
%cd /content/final_project/data/cars_test/
annotsa = loadmat('/content/final_project/data/devkit/cars_meta.mat')
i=1
classes_name=[]
for name in annotsa['class_names'][0]:
  dir_name=str(i)+'_'+str(name[0]).replace(' ','_')
  # os.makedirs(dir_name)
  if dir_name == '174_Ram_C/V_Cargo_Van_Minivan_2012':
    classes_name.append('174_Ram_C')
  else :
    classes_name.append(dir_name)
  i=i+1
classes_name

In [10]:
import shutil
from shutil import copyfile
path_base='/content/final_project/data'
train_data_dir = path_base + '/cars_train'
validation_data_dir = path_base + '/cars_test'


annos = loadmat('/content/final_project/data/devkit/cars_train_annos.mat')
data = [[row.flat[0] for row in line] for line in annos['annotations'][0]]
columns = ['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'class', 'fname']
df_train = pd.DataFrame(data, columns=columns)
gp=df_train.groupby('class')
files=""
for index,classes in gp:
    # print (classes_name[index-1])
    files=classes['fname']
    i=0
    for name,file in files.items():
        # print(file)
        copyfile('{}/{}'.format(train_data_dir,file),'{}/{}/{}'.format(train_data_dir,classes_name[index-1],file))
        if i<25:
          copyfile('{}/{}'.format(validation_data_dir,file),'{}/{}/{}'.format(validation_data_dir,classes_name[index-1],file))
        i=i+1


In [56]:

# train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
#     .flow_from_directory(directory=train_path, target_size=(224,224), classes=['cat', 'dog'], batch_size=10)
train_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(
    directory=train_data_dir,
    target_size=(img_width, img_height),
    class_mode='categorical',
    batch_size=batch_size
    )

test_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)



validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=10,
    class_mode='categorical',
    )

Found 8144 images belonging to 196 classes.
Found 4899 images belonging to 196 classes.


In [57]:

vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in enumerate(vgg16_model.layers):
        layer[1].trainable = False
for layer in vgg16_model.layers[15:]:
            layer.trainable = True
#flatten the results from conv block
x = Flatten()(vgg16_model.output)

#add another fully connected layers with batch norm and dropout
x = Dense(4096, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)

#add another fully connected layers with batch norm and dropout
x = Dense(4096, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)

#add logistic layer with all car classes
predictions = Dense(n_classes, activation='softmax', kernel_initializer='random_uniform', bias_initializer='random_uniform', bias_regularizer=regularizers.l2(0.01), name='predictions')(x)

# this is the model we will train
model = Model(inputs=vgg16_model.input, outputs=predictions)




In [60]:

sgd = SGD(lr=0.001, decay=1e-4, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [36]:
early_stop = EarlyStopping(monitor='val_loss', patience=8, verbose=1, min_delta=1e-4)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, min_delta=1e-4)
callbacks_list = [early_stop, reduce_lr]

In [ ]:
model.summary()
len(train_datagen)

In [ ]:
model_history = model.fit(
    train_datagen,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=len(validation_generator) 
    #callbacks=callbacks_list
    )

Epoch 1/10
128/128 [==============================] - 126s 981ms/step - loss: 7.3255 - accuracy: 0.0082 - val_loss: 10.6957 - val_accuracy: 0.0045
Epoch 2/10
 42/128 [========>.....................] - ETA: 53s - loss: 6.7203 - accuracy: 0.0153

In [81]:
predictions = model.predict(x=validation_generator, verbose=0)
np.round(predictions)

KeyboardInterrupt: ignored

In [86]:
def build_model():
    base_model = densenet.DenseNet121(input_shape=(img_width, img_height, 3),
                                     
                                     include_top=False,
                                     pooling='avg')
    for layer in base_model.layers:
      layer.trainable = True

    x = base_model.output
    x = Dense(1000, kernel_regularizer=regularizers.l1_l2(0.01), activity_regularizer=regularizers.l2(0.01))(x)
    x = Activation('relu')(x)
    x = Dense(500, kernel_regularizer=regularizers.l1_l2(0.01), activity_regularizer=regularizers.l2(0.01))(x)
    x = Activation('relu')(x)
    predictions = Dense(n_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    
    return model

In [88]:
model = build_model()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc', 'mse'])

In [89]:
early_stop = EarlyStopping(monitor='val_loss', patience=8, verbose=1, min_delta=1e-4)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, min_delta=1e-4)
callbacks_list = [early_stop, reduce_lr]

In [91]:
model_history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
    # , callbacks=callbacks_list
    )

Epoch 1/10
255/255 [==============================] - 165s 646ms/step - loss: 6.9947 - acc: 0.0083 - mse: 0.0051
Epoch 2/10
255/255 [==============================] - 163s 640ms/step - loss: 6.9991 - acc: 0.0083 - mse: 0.0051
Epoch 3/10
255/255 [==============================] - 164s 644ms/step - loss: 7.0046 - acc: 0.0083 - mse: 0.0051
Epoch 4/10
255/255 [==============================] - 164s 644ms/step - loss: 7.0168 - acc: 0.0083 - mse: 0.0051
Epoch 5/10
255/255 [==============================] - 162s 637ms/step - loss: 7.0331 - acc: 0.0083 - mse: 0.0051
Epoch 6/10
255/255 [==============================] - 161s 632ms/step - loss: 7.0456 - acc: 0.0083 - mse: 0.0051
Epoch 7/10
255/255 [==============================] - 163s 638ms/step - loss: 7.0519 - acc: 0.0083 - mse: 0.0051
Epoch 8/10
255/255 [==============================] - 162s 637ms/step - loss: 7.0549 - acc: 0.0083 - mse: 0.0051
Epoch 9/10
255/255 [==============================] - 162s 634ms/step - loss: 7.0563 - acc: 0.00

KeyboardInterrupt: ignored